In [1]:
import pandas as pd
import folium

In [2]:
daten=pd.read_csv('Points.csv', sep=';')

In [3]:
def polygon_df_to_geojson(df, properties, lat='Latitude', lon='Longitude'):
    """
    Turn a dataframe containing polygon (by "Polygon Nr") data into a geojson formatted python dictionary

    df : the dataframe to convert to geojson
    properties : a list of columns in the dataframe to turn into geojson feature properties
    lat : the name of the column in the dataframe that contains latitude data
    lon : the name of the column in the dataframe that contains longitude data
    """

    # find polygon groups
    group = df.groupby(['Polygon Nr']).groups.keys()

    # create a new python dict to contain our geojson data, using geojson format
    geojson = {"type": "FeatureCollection", "features": []}

    for i in range(len(group)):
        new_df = df.loc[df['Polygon Nr'] == [x for x in df.groupby(['Polygon Nr']).groups.keys()][i]]
        res_df = new_df.reset_index(drop=True)
        coords = []
        # create a feature template to fill in
        feature = {"type": "Feature",
                   "properties": {},
                   "geometry": {"type": "Polygon",
                                "coordinates": []}}
        # loop through each row in the dataframe and convert each row to geojson format
        for j in range(len(res_df)):
            coords.append([res_df[lon][j], res_df[lat][j]])

        # fill in the coordinates
        feature["geometry"]["coordinates"] = [coords]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = res_df[prop][i]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson["features"].append(feature)

    return geojson

In [4]:
daten=pd.read_csv('Points.csv', sep=';')
daten.head(10)

,Point,Polygon Nr,Longitude,Latitude,Name
0,Point 1,1,6.106033,50.768850,Neumarkt
1,Point 2,1,6.106242,50.768121,Neumarkt
2,Point 3,1,6.106816,50.768195,Neumarkt
3,Point 4,1,6.106580,50.768925,Neumarkt
4,Point 5,1,6.106033,50.768850,Neumarkt
5,Point 6,2,6.102342,50.768327,Frankenberger Park
6,Point 7,2,6.102873,50.767632,Frankenberger Park
7,Point 8,2,6.103136,50.767601,Frankenberger Park
8,Point 9,2,6.103120,50.767452,Frankenberger Park
9,Point 10,2,6.104118,50.767439,Frankenberger Park


In [5]:
m = folium.Map(
    location=[50.77,6.10],
    zoom_start=15,
    #tiles='Mapbox Bright'
)
folium.GeoJson(polygon_df_to_geojson(daten, ['Name']),
    style_function=lambda x: {
        'color' : 'blue',
        'opacity': 0.6,
        'fillColor' : 'blue',
        }).add_to(m)
m

In [6]:
def write_to_file(data,filename):
    f= open(filename,"w+")
    f.write(str(data).replace("'",'"'))
    f.close
    print('{} successfully written to {}'.format(data,filename))

In [7]:
write_to_file(polygon_df_to_geojson(daten, ['Name']),'Out_geojson.txt')

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'Name': 'Neumarkt'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[6.10603272914886, 50.768849944661], [6.106241941452019, 50.76812050224029], [6.10681593418121, 50.7681951433824], [6.1065798997879, 50.7689245846393], [6.10603272914886, 50.768849944661]]]}}, {'type': 'Feature', 'properties': {'Name': 'Frankenberger Park'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[6.10234200954437, 50.7683274614781], [6.10287308692932, 50.767631939096205], [6.103135943412781, 50.767601403730296], [6.1031198501586905, 50.7674521194322], [6.104117631912231, 50.7674385481088], [6.104262471199029, 50.767747294743096], [6.104112267494201, 50.767791401238796], [6.104101538658139, 50.7685751328052], [6.10234200954437, 50.7683274614781]]]}}, {'type': 'Feature', 'properties': {'Name': 'Boendgen'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[6.0994720458984375, 50.766939799241506], [6.098914146423339, 50.7665055101829

In [8]:
dats=pd.read_json('Out_geojson.txt'); dats

,type,features
0,FeatureCollection,"{'type': 'Feature', 'properties': {'Name': 'Ne..."
1,FeatureCollection,"{'type': 'Feature', 'properties': {'Name': 'Fr..."
2,FeatureCollection,"{'type': 'Feature', 'properties': {'Name': 'Bo..."
